In [198]:
import os
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from torch import Tensor
import torchvision
import torch.optim as optim
from tqdm import tqdm
# import torchaudio

In [199]:
mylist_1= os.listdir('data/')
type(mylist_1)
mylist=[]
for item in mylist_1:
    if item[-3:]=='wav':
        mylist.append(item)

In [200]:
class ETRIDataset():
    def __init__(self, file_list):
        self.file_list = file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        ## MFCC
        audio_path = self.file_list[index]
        X, sample_rate = librosa.load('data/'+audio_path, res_type='kaiser_fast',duration=2.5,sr=16000,offset=0.0)
        signal = np.zeros((int(sample_rate *3,)))
        signal[:len(X)] = X
        sample_rate = sample_rate
        mfccs = librosa.feature.mfcc(y=signal, 
                                            sr=sample_rate, 
                                            n_mfcc=32,
                                            )
        mfccs = torch.Tensor(mfccs)
        mfccs = mfccs.unsqueeze(0)
        
        
        ## get label
        if audio_path[-3:] == 'wav':
            if audio_path[7:8] =='a':
                label = 0
            elif audio_path[7:8] =='n':
                label = 1
            elif audio_path[7:8] =='s':
                label = 2
            elif audio_path[7:8] =='h':
                label = 3
        else:
            label=None

        return mfccs, label

In [201]:
class Testwav():
    def __init__(self, file):
        self.file = file
    
    def __len__(self):
        return len(self.file)
    
    def __getitem__(self):
        ## MFCC
        audio_path = self.file
        X, sample_rate = librosa.load(audio_path, res_type='kaiser_fast',duration=2.5,sr=16000,offset=0.0)
        signal = np.zeros((int(sample_rate *3,)))
        signal[:len(X)] = X
        sample_rate = sample_rate
        mfccs = librosa.feature.mfcc(y=signal, 
                                            sr=sample_rate, 
                                            n_mfcc=32,
                                            )
        mfccs = torch.Tensor(mfccs)
        mfccs = mfccs.unsqueeze(0)
        mfccs = mfccs.unsqueeze(0)

        return mfccs

In [202]:
# train,test split
train_size = int(0.8*len(mylist))
test_size = int(len(mylist)-train_size)
train_set, val_set = torch.utils.data.random_split(ETRIDataset(mylist),[train_size,test_size])

In [203]:
# set batch_size
batch_size = 16
# dataloader
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)
dataloaders_dict ={'train':train_dataloader, 'val': val_dataloader}
# test
batch_iterator = iter(dataloaders_dict['train'])
inputs, labels = next(batch_iterator)
print(inputs.size())
print(labels)

torch.Size([16, 1, 32, 94])
tensor([2, 3, 0, 1, 0, 2, 2, 3, 2, 2, 0, 1, 0, 0, 1, 1])


In [204]:
ETRIDataset(mylist).__getitem__(index)[0].size()

torch.Size([1, 32, 94])

In [205]:
# create model
model = torchvision.models.densenet121(pretrained=False)
model.to(torch.device('cuda'))
first_conv_layer = [nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
first_conv_layer.extend(list(model.features))  
model.features= nn.Sequential(*first_conv_layer )  
model.classifier = nn.Linear(in_features=1024, out_features=4)
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters() ,lr=0.00001, weight_decay=1e-6, momentum=0.9)
model.train()

DenseNet(
  (features): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (5): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affin

In [206]:
def train(net, dataloader, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)
    model.to(device)
    torch.backends.cudnn.benchmark = True

    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1,num_epochs))
        print('-------------------------------')

        for phase in ['train','val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and(phase == 'train'):
                continue
            for inputs, labels in tqdm(dataloader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs,1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() *inputs.size(0)

                    epoch_corrects += torch.sum(preds == labels.data)

                    epoch_loss = epoch_loss / len(dataloader[phase].dataset)
                    epoch_acc = epoch_corrects.double() / len(dataloader[phase].dataset)


            print('{} Loss: {:.4f} ACC {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
                

In [207]:
train(model, dataloaders_dict, criterion,optimizer, num_epochs=200)

cuda:0
Epoch 1/2
-------------------------------



 61%|██████    | 22/36 [00:02<00:01,  9.27it/s]


KeyboardInterrupt: 

In [197]:
# save weight
save_path = './saved_models/DenseNet121.pth'
torch.save(model.state_dict(),save_path)